In [1]:
import geopandas as gpd

In [2]:
import numpy as np

In [3]:
shp = gpd.read_file('WB_C3_citiesfixed.shp')

In [4]:
area = shp.geometry.area
perimeter = shp.geometry.length

In [5]:
pp = 4 * np.pi * area / perimeter ** 2

In [6]:
shp['pp'] = pp

In [7]:
shp.to_file('WB_C3_citiesfixed_withpp.shp')

In [10]:
high_pp = pp <= 0.005

In [11]:
high_pp.sum()

133

In [12]:
shp = shp[high_pp]

Buffer and unbuffer to obtain separated polygons and remove tiny polygons.

In [15]:
buffered = shp.buffer(-30 * 5).buffer(30 * 6)

In [18]:
buffered.to_file('WB_C3_buff1.shp')

Subtract this from the original.

In [66]:
subtracted = gpd.overlay(shp, gpd.GeoDataFrame(geometry=[buffered.unary_union], crs='EPSG:3577'), how='difference')

Pull out all the separate subtracted polygons.

In [80]:
subtracted_polygons = []
for p in subtracted.geometry:
    try:  # Multipolygon
        subtracted_polygons.extend(p)
    except TypeError:  # Polygon
        subtracted_polygons.append(p)
subtracted_polygons = gpd.GeoDataFrame(geometry=subtracted_polygons, crs='EPSG:3577')

/tmp/ipykernel_931/773641567.py:4: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  subtracted_polygons.extend(p)
/tmp/ipykernel_931/773641567.py:4: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  subtracted_polygons.extend(p)


In [82]:
subtracted_polygons = subtracted_polygons[subtracted_polygons.area >= 5 * 30 ** 2]

Then remove these polygons and see what's left.

In [89]:
resubtracted = gpd.overlay(shp, subtracted_polygons, how='difference')

In [90]:
resubtracted_polygons = []
for p in resubtracted.geometry:
    try:  # Multipolygon
        resubtracted_polygons.extend(p)
    except TypeError:  # Polygon
        resubtracted_polygons.append(p)
resubtracted_polygons = gpd.GeoDataFrame(geometry=resubtracted_polygons, crs='EPSG:3577')

/tmp/ipykernel_931/412555037.py:4: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  resubtracted_polygons.extend(p)
/tmp/ipykernel_931/412555037.py:4: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  resubtracted_polygons.extend(p)


In [91]:
resubtracted_polygons.to_file('WB_C3_resubtracted.shp')

There are little bits that should be attached but aren't. Find all small polygons attached to these big new polygons. For each big new polygon, search for touching polygons and squish them together.

In [102]:
new_big_geometry = []
unassigned = np.ones(len(subtracted_polygons), dtype=bool)
for i, big in resubtracted_polygons.iterrows():
    mask = subtracted_polygons.exterior.intersects(big.geometry.exterior) & unassigned
    neighbours = subtracted_polygons[mask]
    unassigned[mask] = 0
    if len(neighbours):
        big = big.geometry.union(neighbours.unary_union)
    else:
        big = big.geometry
    new_big_geometry.append(big.buffer(0))

big_geometry = gpd.GeoDataFrame(geometry=new_big_geometry)

In [104]:
big_geometry.to_file('WB_C3_split.shp')